**tensorflow**



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])

In [ ]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000,10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200,10))
# model.fit(train_x, train_y, epochs=10, batch_size=100,
#           validation_data=(val_x, val_y))

# 自定义的数据集
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(100)
dataset = dataset.repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(100)
val_dataset = val_dataset.repeat()
model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset, validation_steps=3)

model.summary()

In [ ]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)

**pytorch**

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
import torch.nn as nn
import torchmetrics
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
print(torch.__version__)

In [ ]:
net = nn.Sequential(nn.Linear(72, 32), nn.ReLU(), nn.Linear(32, 32), nn.ReLU(), nn.Linear(32, 10)) 

# def data_iter(batch_size, features, labels):
#   num_examples = len(features)
#   indices = list(range(num_examples))
#   random.shuffle(indices)
#   for i in range(0, num_examples, batch_size):
#     batch_indices = torch.tensor(indices[i:min(i+batch_size, num_examples)])
#     yield features[batch_indices], labels[batch_indices]

# 如何读到一条数据
class CustomDataSet(Dataset):
    def __init__(self, x, y, transform=None, target_transform=None):
         self.features = x
         self.labels = y

    def __len__(self):
         return len(self.features)

    def __getitem__(self, idx):
         #  self.features = torch.autograd.Variable(self.features, requires_grad = False)
         #  self.features= self.features.data

         # 上述两行的替代方法为self.features.detach_()， 也就是将数据的梯度信息去掉
         self.features.detach_()
         return self.features[idx], self.labels[idx]


In [ ]:
loss = nn.CrossEntropyLoss()

optim =  torch.optim.Adam(net.parameters(), lr=0.001)

# 准备数据
train_x = torch.rand((1000, 72), requires_grad=True)
# train_x = torch.rand((1000, 72))  # 如果train_x不追踪梯度，则self.features = torch.autograd.Variable(self.features, requires_grad = False) 和 self.features= self.features.data可以去掉
train_y = torch.empty(1000, dtype=torch.long).random_(10)



val_x = torch.rand((200, 72))
val_y = torch.empty(200, dtype=torch.long).random_(10)

# 如何以batch形式读到数据
dataset = CustomDataSet(train_x, train_y)
train_dataloader = DataLoader(dataset = dataset, batch_size=100, shuffle=True, num_workers=2)

num_epochs = 10
metric = torchmetrics.Accuracy()

if __name__ == '__main__':
  for epoch in range(num_epochs):
    # for X, y in train_dataloader:
    for i, data in enumerate(train_dataloader, 0):  # 上下两个循环等价
      X, y = data  # 如果采用上一行循环，注释掉该句
      y_hat = net(X)
      l = loss(y_hat, y)
      optim.zero_grad()
      l.backward()
      # print(net[0].weight.grad)
      optim.step()
      acc = metric(y_hat.softmax(dim = -1), y)
    print(f"loss {l.detach().item()}, accracy {acc.detach().item()}. ")

In [ ]:
test_x = torch.rand((1000, 72))
test_y = torch.empty(1000, dtype=torch.long).random_(10)

num_epochs = 10
for epoch in range(num_epochs):
  test_y_hat = net(test_x)
  acc = metric(test_y_hat.softmax(dim = -1), test_y)
  print(f"accracy {acc.detach().item()}. ")